In [134]:
%load_ext autoreload
%autoreload 2
    
from classes.song import Song
from classes.playlist import Playlist
from classes.spotify_manager import SpotifyManager
from classes.lyrics_manager import LyricsManager
from dotenv import load_dotenv
from classes.utils import gather_data_from_folders
import pandas as pd
import os
import seaborn as sns
from classes.text_features import TextFeatureExtractor
from classes.audio_features import AudioFeatureExtractor
from classes.utils import find_songs_to_drop, clean_songs_to_drop
from classes.feature_processor import FeatureProcessor

load_dotenv();

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading the data

In [135]:
playlists_dir = 'audio_sentiment_classifier/playlists'
playlists_dir = os.path.join("/".join(os.path.abspath(os.path.realpath(__name__)).split("/")[:-1]), "playlists")
df_from_folders = gather_data_from_folders(playlists_dir)

INFO: Loading CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/Best Of Global Hip-Hop 2024/Best Of Global Hip-Hop 2024.csv
INFO: Loading CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/HOUSE MIX 2024 🔥 HOUSE PARTY 2024 🔥HOUSE MUSIC 2024 🔥/HOUSE MIX 2024 🔥 HOUSE PARTY 2024 🔥HOUSE MUSIC 2024 🔥.csv
INFO: Loading CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/✨LATINO✨/✨LATINO✨.csv
INFO: Loading CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/final download 1/final download 1.csv
INFO: Loading CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/60s-70s Mix/60s-70s Mix.csv
INFO: Loading CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/Soul Jazz Mix/Soul Jazz Mix.csv
INFO: Loading CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/Alternative Mix/Alternative Mix.csv
INFO: L

In [136]:
df_from_folders.shape

(6152, 24)

In [137]:
df = pd.read_pickle(os.path.join("datasets", "final_data.pkl"))

In [138]:
df.shape

(5422, 122)

In [139]:
df_to_process = df_from_folders[~df_from_folders['id'].isin(df['id'])]

In [140]:
df_to_process.shape

(504, 24)

In [141]:
df_to_process.isna().sum()

id                     0
title                  0
artist                 0
album_art_url          0
popularity            10
explicit               0
album_release_year     0
duration_ms            0
genres                 0
lyrics                 0
mp3_path               0
csv_path               0
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
time_signature         0
dtype: int64

# Getting rid of unusable songs

In [142]:
df_songs_to_drop = find_songs_to_drop(df_to_process, allow_nan_cols=['popularity'])

/home/krystian/Python/Jupyter-projects/thesis/songs_analysis/classes/utils.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incorrect_genre_rows = df_to_process[(df_to_process.genre.isna()) | (df_to_process.genre == 'None')].index


In [143]:
print("Songs that will be dropped: ", len(df_songs_to_drop))

Songs that will be dropped:  38


In [144]:
df_songs_to_drop.head()

,id,title,artist,album_art_url,popularity,explicit,album_release_year,duration_ms,genres,lyrics,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,genre
129,5uuNPORdHV6PYNclkvNGaA,loquita,claydee,https://i.scdn.co/image/ab67616d0000b273718e2c...,28.0,False,2019.0,170723.0,[],"[Letra de ""Loquita""]\n[Coro: Claydee & \nEleni...",...,-5.262,1.0,0.0385,0.1870,0.003380,0.1530,0.716,172.027,4.0,None
150,1kWwNpx68BMAjJeksmN3JV,abusadamente,mc gustta,https://i.scdn.co/image/ab67616d0000b273f66824...,62.0,True,2017.0,215944.0,"['funk carioca', 'funk ostentacao']","[Letra de ""Abusadamente""]\n[Intro: MC DG e \nM...",...,-3.585,0.0,0.1260,0.8090,0.002520,0.2250,0.404,130.079,4.0,None
152,14bJVacqCwgK7QhEe5mFZw,la habana,pinto “wahin”,https://i.scdn.co/image/ab67616d0000b273ec967a...,42.0,False,2017.0,164706.0,[],(La Habana tiene su cosa\nEs aquí donde se vac...,...,-3.007,0.0,0.0438,0.1190,0.000003,0.0823,0.554,108.036,4.0,None
159,6rDWCP2cWEDHpBnK2L14u5,single ladies,remady & manu,https://i.scdn.co/image/ab67616d0000b27346170e...,40.0,False,2012.0,221933.0,[],"""'Manu""'\nHey single ladies, I know you like t...",...,-6.268,0.0,0.0376,0.0201,0.000000,0.4040,0.675,127.968,4.0,None
1598,4GUaG3df9nbrj77GKEB76j,forever you and me,the teskey brothers,https://i.scdn.co/image/ab67616d0000b273958fdc...,74.0,False,2018.0,175592.0,['australian americana'],Forever you and me\nI know my darling I'll be ...,...,-4.940,1.0,0.0331,0.4360,0.000010,0.0971,0.538,175.159,3.0,None


In [145]:
clean_songs_to_drop(df_songs_to_drop)

INFO: Removed MP3 file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/✨LATINO✨/loquita.mp3
INFO: Updated CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/✨LATINO✨/✨LATINO✨.csv
INFO: Removed MP3 file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/✨LATINO✨/abusadamente.mp3
INFO: Updated CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/✨LATINO✨/✨LATINO✨.csv
INFO: Removed MP3 file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/✨LATINO✨/la habana.mp3
INFO: Updated CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/✨LATINO✨/✨LATINO✨.csv
INFO: Removed MP3 file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/✨LATINO✨/single ladies.mp3
INFO: Updated CSV file: /home/krystian/Python/Jupyter-projects/thesis/songs_analysis/playlists/✨LATINO✨/✨LATINO✨.csv
INFO: Removed MP3 file: /home/krystian/Python/Jupyter-p

In [146]:
df_to_process = df_to_process[~df_to_process.id.isin(df_songs_to_drop.id)]

In [147]:
df_to_process.shape[0]

466

In [148]:
df_to_process.to_pickle(os.path.join("datasets", "onboarding_step1.pkl"))

---

# Adding audio features

In [22]:
aud_processor = FeatureProcessor(
    extractor=AudioFeatureExtractor(),
    input_file=os.path.join("datasets", "onboarding_step1.pkl"),
    output_file=os.path.join('datasets', "onboarding_step2.pkl"),
    batch_size=10
)
aud_processor.process_batches()

Total records to process: 281
Processing in batches of size 10...


Processing Batches:   0%|          | 0/29 [00:00<?, ?batch/s]

Extracting Audio Features:   0%|          | 0/10 [00:00<?, ?file/s]

Saved batch with 10 records to datasets/onboarding_step2.pkl


Extracting Audio Features:   0%|          | 0/10 [00:00<?, ?file/s]

...

Saved batch with 10 records to datasets/onboarding_step2.pkl


Extracting Audio Features:   0%|          | 0/10 [00:00<?, ?file/s]

Saved batch with 10 records to datasets/onboarding_step2.pkl


Extracting Audio Features:   0%|          | 0/1 [00:00<?, ?file/s]

Saved batch with 1 records to datasets/onboarding_step2.pkl


# Adding textual features

### Combining existing data with data being onboarded to calculate tfidf on the whole corpus

In [35]:
text_features = ['word_count',
 'unique_word_count',
 'avg_word_length',
 'syllable_count',
 'sentiment_polarity',
 'sentiment_subjectivity',
 'readability_score',
 'noun_ratio',
 'verb_ratio',
 'vader_compound',
 'repetition_count',
 'average_syllables_per_word',
 'language',
 'genre',
 'word_count',
 'unique_word_count',
 'avg_word_length',
 'syllable_count',
 'sentiment_polarity',
 'sentiment_subjectivity',
 'readability_score',
 'vader_compound',
 'repetition_count',
 'average_syllables_per_word',
 'language'] 

In [36]:
df_complete_data = pd.read_pickle(os.path.join("datasets", "final_data.pkl"))
df_complete_data.drop([c for c in df_complete_data.columns if c.startswith('tfidf') or c in text_features ], axis=1, inplace=True)

In [37]:
df_complete_data.lyrics

0       come skinni love last year pour littl salt nev...
1       life fleet slow chill us wave feed bone right ...
2       id give forev touch caus know feel somehow you...
3       freedom within freedom without tri catch delug...
4       know youv suffer dont want hide cold loveless ...
                              ...                        
5149    must monday dumb day cant drag butt outta bed ...
5150    god know hide weak drunken heart guess kiss gi...
5151    mouth revolv fire bullet sky love like soldier...
5152    see your look find smile face peac mind state ...
5153    buri secret skin come away innoc leav sin air ...
Name: lyrics, Length: 5154, dtype: object

In [38]:
df_onboarding = pd.read_pickle(os.path.join("datasets", "onboarding_step2.pkl"))
df_onboarding.drop([c for c in df_onboarding.columns if c.startswith('tfidf') or c in text_features ], axis=1, inplace=True)
df_onboarding.head()

,id,title,artist,album_art_url,popularity,explicit,album_release_year,duration_ms,genres,lyrics,...,chroma_12,spectral_contrast_1,spectral_contrast_2,spectral_contrast_3,spectral_contrast_4,spectral_contrast_5,spectral_contrast_6,spectral_contrast_7,tempo_extracted,zcr
0,4Pt80KvnL6e8DCd9QVx7Ht,pink lemonade,leostaytrill,https://i.scdn.co/image/ab67616d0000b273873f0e...,70.0,True,2024.0,122208.0,[],"[Intro]\n(\nXeretto, fuck you doin', man?\n)\n...",...,0.465013,15.299487,11.837251,16.164133,18.430578,18.852168,18.879488,51.501343,144.230769,0.041279
1,0Sr7ssScx54yxdM2ols7nW,q u e v a s h a c e r h o y,omar courtz,https://i.scdn.co/image/ab67616d0000b273996764...,88.0,True,2024.0,224022.0,"['trap boricua', 'trap latino', 'urbano latino']","[Letra de ""Que Vas Hacer Hoy?""]\n[Parte I]\n[I...",...,0.359806,16.174883,11.482940,14.663843,18.272432,18.802098,17.797354,53.469670,119.680851,0.052631
2,2hrycoFU1mZw6YPvMcn8yC,kehlani,jordan adetunji,https://i.scdn.co/image/ab67616d0000b273f2cd13...,84.0,False,2024.0,122360.0,['irish hip hop'],"[Intro: Summer Walker]\n(\nWock\n)\nYou got, y...",...,0.527515,14.913413,9.912072,13.995783,15.869135,17.789146,17.671408,53.769190,106.132075,0.036518
3,0OA00aPt3BV10qeMIs3meW,big dawgs,hanumankind,https://i.scdn.co/image/ab67616d0000b273d9afe5...,84.0,True,2024.0,190666.0,"['desi hip hop', 'indian underground rap', 'ma...","[Intro]\nUh, yeah, uh, yeah\nUh, yeah, uh, yea...",...,0.536583,14.527773,10.739625,13.807236,15.382049,17.405447,16.539265,56.109247,119.680851,0.056914
4,0io16MKpbeDIdYzmGpQaES,embrace it,ndotz,https://i.scdn.co/image/ab67616d0000b273536637...,86.0,True,2024.0,104418.0,['chill drill'],"[Chorus]\nShe bad, mm-hmm, I like this\n (Yeah...",...,0.456958,12.556766,11.990558,14.992236,16.226411,18.184162,19.636641,54.326191,114.795918,0.035092


In [39]:
df_complete_data.shape

(5154, 58)

In [40]:
df_onboarding.shape

(281, 58)

In [44]:
df_onboarding['lyrics'] = df_onboarding.lyrics.apply(lambda x: TextFeatureExtractor.preprocess_text(x))

In [45]:
df_final = pd.concat([df_complete_data, df_onboarding])
df_final

,id,title,artist,album_art_url,popularity,explicit,album_release_year,duration_ms,genres,lyrics,...,spectral_contrast_1,spectral_contrast_2,spectral_contrast_3,spectral_contrast_4,spectral_contrast_5,spectral_contrast_6,spectral_contrast_7,tempo_extracted,zcr,csv_path
0,4RL77hMWUq35NYnPLXBpih,skinny love,birdy,https://i.scdn.co/image/ab67616d0000b2733661c0...,70.0,0,2011.0,201080.0,"['neo mellow', 'uk pop', 'viral pop']",come skinni love last year pour littl salt nev...,...,17.627332,13.771729,19.275149,22.244659,23.802122,22.563887,46.881367,119.680851,0.031446,/home/krystian/Python/Jupyter-projects/thesis/...
1,5PsjZ21f5tMPFf1sJpokm7,deathwish,red sun rising,https://i.scdn.co/image/ab67616d0000b273612f16...,NaN,0,2018.0,287946.0,"['modern hard rock', 'post-grunge']",life fleet slow chill us wave feed bone right ...,...,16.215804,10.309580,13.744232,16.058571,17.278253,16.887822,53.087236,133.928571,0.051242,/home/krystian/Python/Jupyter-projects/thesis/...
2,6vrUTGn5p8IrfTZ0J6sIVM,iris,the goo goo dolls,https://i.scdn.co/image/ab67616d0000b273d54c4b...,71.0,0,2007.0,289906.0,"['neo mellow', 'permanent wave', 'pop rock', '...",id give forev touch caus know feel somehow you...,...,15.357601,10.201447,13.698081,17.050868,18.783636,17.473927,56.185155,152.027027,0.069683,/home/krystian/Python/Jupyter-projects/thesis/...
3,0Leo4GP0UKG4tUABm9JATe,don't dream it's over,crowded house,https://i.scdn.co/image/ab67616d0000b27380ed24...,NaN,0,2017.0,231377.0,"['australian rock', 'permanent wave']",freedom within freedom without tri catch delug...,...,14.941456,11.061548,15.084157,17.604808,18.362215,18.056211,51.318052,160.714286,0.044647,/home/krystian/Python/Jupyter-projects/thesis/...
4,0It6VJoMAare1zdV2wxqZq,undisclosed desires,muse,https://i.scdn.co/image/ab67616d0000b273b6d456...,65.0,0,2009.0,235000.0,"['alternative rock', 'modern rock', 'permanent...",know youv suffer dont want hide cold loveless ...,...,14.934434,11.366229,15.412446,16.330238,17.806636,17.648626,53.669779,117.187500,0.041844,/home/krystian/Python/Jupyter-projects/thesis/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,5PycBIeabfvX3n9ILG7Vrv,propuesta indecente,romeo santos,https://i.scdn.co/image/ab67616d0000b273e9da42...,80.0,False,2014.0,235133.0,"['bachata', 'latin hip hop', 'latin pop', 'tra...",hola llaman romeo es un placer conocerla que b...,...,13.910094,12.284040,15.973888,18.085161,20.063634,17.834683,55.445500,122.282609,0.051709,/home/krystian/Python/Jupyter-projects/thesis/...
277,228BxWXUYQPJrJYHDLOHkj,gasolina,daddy yankee,https://i.scdn.co/image/ab67616d0000b2736bdcdf...,81.0,False,2004.0,192600.0,"['latin hip hop', 'reggaeton', 'trap latino', ...",zumbal mambo pa ke mi gata prendan los motor z...,...,14.630034,11.143904,14.681024,16.917971,18.737248,17.295369,53.449685,95.338983,0.048868,/home/krystian/Python/Jupyter-projects/thesis/...
278,059bcIhyc2SBwm6sw2AZzd,te vi,piso 21,https://i.scdn.co/image/ab67616d0000b273244a59...,78.0,False,2018.0,231848.0,"['colombian pop', 'latin pop', 'reggaeton', 'r...",letra de te vi ft micro tdh intro micro tdh el...,...,17.848964,12.884122,15.739994,17.014210,18.715929,17.977095,53.879891,102.272727,0.038930,/home/krystian/Python/Jupyter-projects/thesis/...
279,23WI5V2eD4EyGKxSl7Pyeq,andas en mi cabeza,chino & nacho,https://i.scdn.co/image/ab67616d0000b2736df5cc...,72.0,False,2016.0,247493.0,"['electro latino', 'latin hip hop', 'latin pop...",letra de anda en mi cabeza ft daddi yanke intr...,...,14.107049,10.202836,13.970952,16.670884,18.625395,17.422064,55.418099,104.166667,0.055295,/home/krystian/Python/Jupyter-projects/thesis/...


In [46]:
# df_final['lyrics'] = df_final.lyrics.apply(lambda x: TextFeatureExtractor.preprocess_text(x))

In [47]:
df_final.to_pickle(os.path.join("datasets", "onboarding_step3.pkl"))

### Calculating textual features

In [48]:
tf_processor = FeatureProcessor(
    extractor=TextFeatureExtractor(n_pca_components=50),
    input_file=os.path.join("datasets", "onboarding_step3.pkl"),
    output_file=os.path.join('datasets', "onboarding_step4.pkl"),
    batch_size=None,
    overwrite_existing=True
)
tf_processor.process_batches()

Total records to process: 5435
Processing in a single batch without multiprocessing...
Saved batch with 5435 records to datasets/onboarding_step4.pkl


# Cleaning

In [90]:
# df = pd.read_pickle(os.path.join("datasets", "onboarding_step4.pkl"))
df = pd.read_pickle(os.path.join("datasets", "final_data.pkl"))
df.shape

(5435, 122)

In [91]:
from classes.constants import GENRE_MAPPING
from classes.utils import reduce_genres_with_regex

df['genre'] = df['genres'].apply(lambda x: reduce_genres_with_regex(eval(x), GENRE_MAPPING))

df['genre'].value_counts()

pop             2525
rock             740
hip hop          685
r&b              255
indie            254
folk             225
dance            202
electronic        88
metal             70
classical         60
latin             59
reggae            53
experimental      51
jazz              50
country           43
alternative       40
grime             22
Name: genre, dtype: int64

In [98]:
incorrect_genre_songs = df[(df.genre.isna()) | (df.genre == 'None')]

In [101]:
print("{} songs have incorrect genre and will be dropped".format(incorrect_genre_songs.shape[0]))
df = df[~df.id.isin(incorrect_genre_songs.id)]

13 songs have incorrect genre and will be dropped


In [102]:
# Fixing the dtypes
cast_to_int_columns = ['duration_ms', 'album_release_year', 'key', 'mode', 'time_signature', 'explicit']
df[cast_to_int_columns] = df[cast_to_int_columns].astype(int)

cast_to_category_columns = ['key', 'mode', 'time_signature', 'explicit', 'language', 'genre']
df[cast_to_category_columns] = df[cast_to_category_columns].astype(str).astype('category')

/home/krystian/Python/VENVS/jupyter310/lib/python3.10/site-packages/pandas/core/arrays/categorical.py:527: RuntimeWarning: invalid value encountered in cast
  fill_value = lib.item_from_zerodim(np.array(np.nan).astype(dtype))
/home/krystian/Python/VENVS/jupyter310/lib/python3.10/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [104]:
df.to_pickle(os.path.join("datasets", 'final_data.pkl'))

In [105]:
for pkl in [ file for file in os.listdir('datasets') if file.startswith('onboarding') and file.endswith('pkl')]:
    os.remove(os.path.join('datasets', pkl))